In [3]:
@file:DependsOn("io.ktor:ktor-client-core:2.0.0")
@file:DependsOn("io.ktor:ktor-client-cio:2.0.0")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:2.0.0")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:2.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
%use dataframe

import ro.jf.bk.account.sdk.AccountSdk
import ro.jf.bk.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk
import java.util.UUID.randomUUID

val ACCOUNTS_CSV_FILE = "../../data/provision/accounts.csv"
val FUNDS_CSV_FILE = "../../data/provision/funds.csv"

val userId = randomUUID()

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()

In [4]:
import ro.jf.bk.account.api.model.CreateCurrencyAccountTO
import ro.jf.bk.fund.api.model.CreateFundTO

val createAccountRequests = DataFrame.readCSV(ACCOUNTS_CSV_FILE, delimiter = ';')
    .map { row -> CreateCurrencyAccountTO(row.getValue<String>("account"), row.getValue<String>("currency")) }
createAccountRequests

[CreateCurrencyAccountTO(name=Cash RON, currency=Cash RON), CreateCurrencyAccountTO(name=ING RON, currency=RON), CreateCurrencyAccountTO(name=ING Economy, currency=RON), CreateCurrencyAccountTO(name=Food Coupons, currency=RON)]

In [13]:
val createFundRequests = DataFrame.readCSV(FUNDS_CSV_FILE, delimiter = ';')
    .map { row -> CreateFundTO(row.getValue<String>("fund"), emptyList()) }
createFundRequests

[CreateFundTO(name=Expenses, accounts=[]), CreateFundTO(name=Developer Income, accounts=[]), CreateFundTO(name=Savings, accounts=[])]

In [11]:
import ro.jf.bk.account.api.model.AccountTO
import kotlinx.coroutines.runBlocking

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(userId)
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = createAccountRequests
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(userId, it) }
    existingAccounts + newAccounts
}
accounts

[Currency(id=ab6e37a1-70b8-43b2-8f37-8215ed4e299b, name=Cash RON, currency=Cash RON), Currency(id=d88d2fb7-3ebb-415c-86a4-35f474ef4d82, name=Food Coupons, currency=RON), Currency(id=9b8d949a-30e5-47cf-b4e9-df2ccdecd4b7, name=ING Economy, currency=RON), Currency(id=af74c3b4-5e0a-4d76-b517-465911ab3c4b, name=ING RON, currency=RON)]

In [17]:
val funds = runBlocking {
    // TODO(Johann) fix npe on get existing funds without accounts
    val existingFunds = fundSdk.listFunds(userId)
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = createFundRequests
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(userId, it) }
    existingFunds + newFunds
}
funds

ro.jf.bk.fund.api.exception.FundApiException.Generic: Internal error.

In [20]:
import kotlinx.coroutines.runBlocking
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*

val csvFile = File("../../data/wallet/2019/wallet_export_2019_01.csv")
val importConfiguration = ImportConfigurationTO(
    fileType = ImportFileTypeTO.WALLET_CSV,
    accountMatchers = listOf(
        AccountMatcherTO("ING Old", "ING RON"),
    )
)
// TODO(Johann) fix npe on import
val response = runBlocking { importSdk.import(userId, csvFile, importConfiguration) }
response

io.ktor.client.call.NoTransformationFoundException: No transformation found: class io.ktor.utils.io.ByteBufferChannel -> class ro.jf.funds.importer.api.model.ImportResponse
with response from http://localhost:5207/bk-api/import/v1/imports:
status: 500 Internal Server Error
response headers: 
Content-Length: 0
